# 一种划分patch的策略
- 用5*5的pathch中最多的个点的像素值取代替一个点像素值，这样会丢失数据的信息

In [19]:
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import numpy as np
%matplotlib inline

- tensorflow中添加每一层的命令都是大写的
- conv kernel 的数量是以 2 的 n 次方的形式递增的
- 通过 pooling 池化层，相同大小的卷积核覆盖的区域将变大，这是pooling的重要意义

In [20]:
#  load the Indian pines dataset which is the .mat format
def loadData():
    data_path = os.path.join(r'E:\Eric_HSI\hyperspectral_datasets')
    data = sio.loadmat(os.path.join(data_path, 'Salinas_corrected.mat'))
    labels = sio.loadmat(os.path.join(data_path, 'Salinas_gt.mat'))
    return data, labels

X, y = loadData()

# 打印keys
print(X.keys())
print(y.keys())

dict_keys(['__header__', '__version__', '__globals__', 'salinas_corrected'])
dict_keys(['__header__', '__version__', '__globals__', 'salinas_gt'])


In [21]:
# 将dict to ndarray
def dict_to_array(X, y):
    data = X[list(X)[-1]]
    labels = y[list(y)[-1]]
    return data, labels

data, data_gt = dict_to_array(X, y)

In [22]:
print(data.shape)
print(data_gt.shape)

(512, 217, 204)
(512, 217)


In [23]:
# 归一化，这个归一化的方法有点不一样
data = data -float(np.min(data))
data = data/np.max(data)

In [24]:
# 原始数据转化为X_data 为 (4205, 5, 5, 204) # 21,445,500; 4386*25=109,650
X_data = []
step = 5
x = range(0, 510, step)
y = range(0, 215, step)
for i in x:
  for j in y:
    X_data.append(data[i:i+step, j:j+step])
print(len(X_data))
# X_data = np.array(X_data)[:4205].astype(np.float32)
X_data = np.array(X_data).astype(np.float32)
print(X_data.shape)

4386
(4386, 5, 5, 204)


In [10]:
# # 学习使用scipy.stats.mode
# #  from scipy.stats import mode
# from scipy.stats import mode
# a = np.array([[2, 2, 2, 1],
#               [1, 2, 2, 2],
#               [1, 1, 3, 3]])
# print("# Print mode(a):", mode(a))
# print("# Print mode(a.transpose()):", mode(a.transpose()))
# print("# a的每一列中最常见的成员为：{}，分别出现了{}次。".format(mode(a)[0][0], mode(a)[1][0]))
# print("# a的第一列中最常见的成员为：{}，出现了{}次。".format(mode(a)[0][0][0], mode(a)[1][0][0]))
# print("# a的每一行中最常见的成员为：{}，分别出现了{}次。".format(mode(a.transpose())[0][0], mode(a.transpose())[1][0]))
# print("# a中最常见的成员为：{}，出现了{}次。".format(mode(a.reshape(-1))[0][0], mode(a.reshape(-1))[1][0]))
# print(mode(a))
# print(mode(a)[0])
# print(mode(a)[1])
# print(mode(a)[0][0])
# print(mode(a)[1][0])
# print(mode(a.reshape(-1)))
# print(mode(a.reshape(-1))[0][0])

In [11]:
# data_gt[0:5, 0:5].reshape(25, -1)
# mode(data_gt[0:5, 0:5].reshape(25, -1))[0][0][0]   # 输出在这25个像素中出现最多的像素的类别

0

In [30]:
# 这一行的作用是把5*5小方块中出现最多的像素作为这一个小方块中的类别编号，作用可能就是降低数据的容量，加快训练速度！！
# 标签数据 Y_data
import scipy.stats
Y_data = []
for i in x:
  for j in y:
      # 返回传入数组/矩阵中最常出现的成员以及出现的次数。
    label = scipy.stats.mode(data_gt[i:i+step, j:j+step].reshape(step*step, 1))[0][0][0]
    Y_data.append(label)
Y_data = np.array(Y_data)
print(Y_data.shape)
type(Y_data)

(4386,)


numpy.ndarray

In [29]:
# # 对比一下用5*5代替1*1的情况下两种标签方式的图像
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(1, 2)
# ax[0].imshow(Y_data.reshape(102, 43), cmap='nipy_spectral')
# ax[1].imshow(data_gt, cmap='nipy_spectral')

In [31]:
Y_data = Y_data.astype(np.int32)
Y_data = Y_data.reshape(len(Y_data), 1)
# fig, ax = plt.subplots(1, 2)
# ax[0].imshow(Y_data.reshape(100, 42), cmap='nipy_spectral')
# ax[1].imshow(data_gt, cmap='nipy_spectral')

In [32]:
print(X_data.shape)
print(Y_data.shape)

(4386, 5, 5, 204)
(4386, 1)


In [33]:
from sklearn.model_selection import train_test_split
(train, test, train_label, test_label) = train_test_split(X_data, Y_data, test_size=0.6)
train.shape, test.shape, train_label.shape, test_label.shape

((1754, 5, 5, 204), (2632, 5, 5, 204), (1754, 1), (2632, 1))

In [34]:
BATCHSZ = 32

In [36]:
# 创建dataset
db = tf.data.Dataset.from_tensor_slices((train,train_label))  # 训练集合标签！
db = db.shuffle(4205).batch(BATCHSZ).repeat()

db_test = tf.data.Dataset.from_tensor_slices((test,test_label))
db_test = db_test.batch(BATCHSZ)
db, db_test

(<RepeatDataset shapes: ((None, 5, 5, 204), (None, 1)), types: (tf.float32, tf.int32)>,
 <BatchDataset shapes: ((None, 5, 5, 204), (None, 1)), types: (tf.float32, tf.int32)>)